In [1]:
# --*-- encoding:utf-8 --*--
import pandas as pd
import numpy as np
import jieba
import re

In [2]:
# loading data
def loadfile():
    neg = pd.read_excel('./neg1.xls', header=None, index=None)
    pos = pd.read_excel('./pos1.xls', header=None, index=None)
    #merge all data
    neg = np.array(neg[0])
    pos = np.array(pos[0])
    return neg,pos

In [3]:
neg, pos = loadfile()

In [4]:
#generating set of disused words
def getstopword(stopwordPath):
    stoplist = set()
    for line in stopwordPath:
        stoplist.add(line.strip())
        # print line.strip()
    return stoplist

In [5]:
#divide the sentence and remove the disused words
def wordsege(text):
    # get disused words set
    stopwordPath = open('./stopwords(ch).txt', 'r')
    stoplist = getstopword(stopwordPath)
    stopwordPath.close()

    # divide the sentence and remove the disused words with jieba,return list
    text_list = []
    for document in text:

        seg_list = jieba.cut(document.strip())
        fenci = []

        for item in seg_list:
            if item not in stoplist and re.match(r'-?\d+\.?\d*', item) == None and len(item.strip()) > 0:
                fenci.append(item)
        # if the word segmentation of the sentence is null,the label of the sentence should be deleted accordingly
        if len(fenci) > 0:
            text_list.append(fenci)
    return text_list

In [6]:
def tokenizer(neg, pos):
    neg_sege = wordsege(neg)
    pos_sege = wordsege(pos)
    combined = np.concatenate((pos_sege,neg_sege))
    # generating label and meging label data
    y = np.concatenate((np.ones(len(pos_sege), dtype=int), np.zeros(len(neg_sege), dtype=int)))
    return combined,y

In [7]:
combined,y = tokenizer(neg, pos)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\chenx\AppData\Local\Temp\jieba.cache
Loading model cost 1.862 seconds.
Prefix dict has been built succesfully.


In [8]:
from sklearn.cross_validation import train_test_split
x_train, x_test, y_train, y_test = train_test_split(combined,y, test_size = 0.3)

C:\Users\chenx\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
from gensim.models import Word2Vec
model = Word2Vec(combined, size=128, window=5, min_count=5, workers=4)

C:\Users\chenx\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
# vec_size 指的是我们本身vector的size
def transform_to_matrix(x, padding_size=256, vec_size=128):
    res = []
    for sen in x:
        matrix = []
        for i in range(padding_size):
            try:
                matrix.append(model[sen[i]].tolist())
            except:
                # 这里有两种except情况，
                # 1. 这个单词找不到
                # 2. sen没那么长
                # 不管哪种情况，我们直接贴上全是0的vec
                matrix.append([0] * vec_size)
        res.append(matrix)
    return res

In [11]:
x_train = transform_to_matrix(x_train)
x_test = transform_to_matrix(x_test)

C:\Users\chenx\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [12]:
# 搞成np的数组，便于处理
x_train = np.array(x_train)
x_test = np.array(x_test)

# 看看数组的大小
print(x_train.shape)
print(x_test.shape)

(5467, 256, 128)
(2344, 256, 128)


In [13]:
from keras import backend as K
K.set_image_dim_ordering('th')

C:\Users\chenx\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
from keras import backend as K
K.set_image_data_format('channels_first')

In [23]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(128, input_dim=128, input_length=256, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

C:\Users\chenx\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  
C:\Users\chenx\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.3, input_shape=(256, 128))`
  


In [16]:
# compile模型
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model.compile(loss='mse',
#               optimizer='adadelta',
#               metrics=['accuracy'])

# 两层LSTM
model.fit(x_train, y_train, batch_size=64, epochs=3, verbose=0)
score = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.6867430869222908
Test accuracy: 0.564419795221843


In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 256, 128)          131584    
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               394240    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 526,081
Trainable params: 526,081
Non-trainable params: 0
_________________________________________________________________


In [27]:
from keras.layers import TimeDistributed 
from keras.layers import Bidirectional, BatchNormalization

In [30]:
model = Sequential() 
input_shape = (256, 128) 
model.add(Bidirectional(LSTM(units=20,return_sequences=False),input_shape=input_shape))
model.add(Dropout(0.5)) 
model.add(BatchNormalization()) 
# model.add(TimeDistributed(Dense(1, activation='sigmoid'))) 
model.add(Dense(1, activation='sigmoid'))
# LSTM参数个数计算：ht-1与xt拼接、隐藏单元数、四个门的bias 
# （20+40）*units*4+20*4 # # 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 

In [31]:
batch_size = 64 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(x_train, y_train, batch_size=batch_size, epochs=3, validation_data=(x_test, y_test), verbose=1)
score, acc = model.evaluate(x_test, y_test,batch_size=batch_size)
print (score, acc)

Train on 5467 samples, validate on 2344 samples
Epoch 1/3
5467/5467 [==============================] - 51s 9ms/step - loss: 0.6386 - acc: 0.6470 - val_loss: 0.5236 - val_acc: 0.7658
Epoch 2/3
5467/5467 [==============================] - 44s 8ms/step - loss: 0.4073 - acc: 0.8356 - val_loss: 0.3696 - val_acc: 0.8490
Epoch 3/3
2344/2344 [==============================] - 5s 2ms/step
0.32361925889200727 0.8775597267590285


In [32]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_7 (Bidirection (None, 40)                23840     
_________________________________________________________________
dropout_2 (Dropout)          (None, 40)                0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 40)                160       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 41        
Total params: 24,041
Trainable params: 23,961
Non-trainable params: 80
_________________________________________________________________


In [ ]:
model = Sequential() 
input_shape = (256, 128) 
model.add(Bidirectional(LSTM(units=20,return_sequences=True),input_shape=input_shape)) 
model.add(Dropout(0.5)) 
model.add(BatchNormalization()) 
model.add(TimeDistributed(Dense(n_classes))) 
model.add(Dropout(0.2)) 
crf = ChainCRF() 
model.add(crf) 
model.compile(loss=crf.loss, optimizer='rmsprop', metrics=['accuracy'])

batch_size = 64 
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(x_train, y_train, batch_size=batch_size, epochs=30, validation_data=(x_test, y_test), verbose=1)
score, acc = model.evaluate(x_test, y_test,batch_size=batch_size)
print (score, acc)